## Titel: Import data
## Author: Achraf Aboukinana

## Beschreibung des Codes

Dieser Code dient der Analyse, Bereinigung und Kombination von Datensätzen, um eine zentrale und bereinigte Datenbasis zu erstellen. Die Hauptaufgaben sind wie folgt:


# Datenimport mit Pandas und Feiertagsbibliothek

Dieses Skript lädt verschiedene Datensätze ein, um sie später für Analysen oder Modellierungsschritte zu verwenden. 

## Importierte Bibliotheken

In [94]:
import pandas as pd

data_pfad= 'kaggel/test.csv'
model_data_pfad = 'final_data_test.csv'
# Import Data
kiwo_data = pd.read_csv('kaggel/kiwo.csv', parse_dates=['Datum'])
umsatzdaten_gekuerzt_data = pd.read_csv(data_pfad, parse_dates=['Datum'])

wetter_data = pd.read_csv('kaggel/wetter.csv', parse_dates=['Datum'])


# Datenzusammenführung und Ergänzung der KielerWoche-Information

Dieses Skript kombiniert Umsatz-, Wetter- und KielerWoche-Daten und ergänzt fehlende Informationen, um eine vollständige Datengrundlage für Analysen oder Modelle zu erstellen.

## Schritte der Datenverarbeitung

### 1. Zusammenführen von Umsatz- und Wetterdaten
Die `umsatzdaten_gekuerzt_data` und `wetter_data` werden auf der gemeinsamen Spalte `Datum` mit einem **Left Join** zusammengeführt.

### 2. Hinzufügen von KielerWoche-Information
Die `kiwo_data` wird mit den bereits zusammengeführten Daten (`merged_data`) auf der Spalte `Datum` kombiniert. Fehlende Werte in `KielerWoche` werden durch `0` ersetzt.


In [73]:
import pandas as pd
import holidays

# Merge Umsatzdaten mit Wetterdaten auf 'Datum'
merged_data = pd.merge(umsatzdaten_gekuerzt_data, wetter_data, on='Datum', how='left')

# Füge KielerWoche-Information hinzu und ersetze fehlende Werte durch 0
final_data = pd.merge(merged_data, kiwo_data, on='Datum', how='left')
final_data['KielerWoche'] = final_data['KielerWoche'].fillna(0).astype(int)


# Umgang mit fehlenden Werten (NaN)

In diesem Abschnitt wird überprüft, wie viele fehlende Werte (`NaN`) in den Daten vorhanden sind, und anschließend werden diese Werte behandelt, um saubere Daten für die Analyse oder Modellierung zu erhalten.

## Schritte der Datenverarbeitung

### 1. **Überprüfung der Anzahl der NaN-Werte**
Mit der Methode `isna().sum()` wird ermittelt, wie viele fehlende Werte (`NaN`) in jeder Spalte des DataFrame `final_data` vorhanden sind. Das Ergebnis wird für jede Spalte separat angezeigt.


In [74]:

# Anzahl der NaN-Werte überprüfen
nan_counts = final_data.isna().sum()
print("Anzahl der NaN-Werte pro Spalte:")
print(nan_counts)

# Entferne Zeilen mit NaN-Werten
clean_data = final_data.dropna()
print(len(clean_data))


Anzahl der NaN-Werte pro Spalte:
id                       0
Datum                    0
Warengruppe              0
Bewoelkung              65
Temperatur              65
Windgeschwindigkeit     65
Wettercode             337
KielerWoche              0
dtype: int64
1493


# Hinzufügen zeitabhängiger Variablen

In diesem Abschnitt werden zeitbezogene Merkmale zu den Daten hinzugefügt. Diese Variablen können in Analysen oder Modellierungen hilfreich sein, um saisonale oder zeitliche Muster zu erkennen.

## Schritte der Datenverarbeitung

### 1. **Wochentag hinzufügen**
Die Spalte `Wochentag` wird basierend auf der Spalte `Datum` berechnet. 
- **Wertebereich**: Montag = 0, Dienstag = 1, ..., Sonntag = 6.


In [75]:

# **Zeitabhängige Variablen hinzufügen**
clean_data['Wochentag'] = clean_data['Datum'].dt.dayofweek  # Montag=0, Sonntag=6
clean_data['Monat'] = clean_data['Datum'].dt.month
clean_data['Jahr'] = clean_data['Datum'].dt.year

clean_data['Jahreszeit'] = clean_data['Datum'].dt.month % 12 // 3 + 1  # Frühling=1, Sommer=2, Herbst=3, Winter=4


C:\Users\lorda\AppData\Local\Temp\ipykernel_24476\1988144829.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['Wochentag'] = clean_data['Datum'].dt.dayofweek  # Montag=0, Sonntag=6
C:\Users\lorda\AppData\Local\Temp\ipykernel_24476\1988144829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['Monat'] = clean_data['Datum'].dt.month
C:\Users\lorda\AppData\Local\Temp\ipykernel_24476\1988144829.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

# Hinzufügen von Jahreszeiten, Temperaturempfindungen und Dummy-Variablen

Dieser Abschnitt erweitert die Daten um zusätzliche Features basierend auf Jahreszeiten und Temperatur, gefolgt von einem Dummy-Encoding für die Temperaturkategorien.

## Schritte der Datenverarbeitung

### 1. **Bestimmung der Jahreszeit**
Eine Funktion `get_season` wird definiert, um basierend auf dem Monat (`Monat`) die entsprechende Jahreszeit zu bestimmen:
- **Winter**: Dezember, Januar, Februar,
- **Frühling**: März, April, Mai,
- **Sommer**: Juni, Juli, August,
- **Herbst**: September, Oktober, November.

Die Jahreszeit wird in der neuen Spalte `Jahreszeit` gespeichert.


In [76]:



# Jahreszeit bestimmen
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Frühling"
    elif month in [6, 7, 8]:
        return "Sommer"
    elif month in [9, 10, 11]:
        return "Herbst"

clean_data['Jahreszeit'] = clean_data['Monat'].apply(get_season)

# Gefühl abhängig von Temperatur und Jahreszeit
def temperature_feeling(temp, season):
    if season == "Winter":
        if temp <= 0:
            return "Kalt"
        elif temp <= 10:
            return "Mild"
        else:
            return "Warm"
    elif season == "Frühling":
        if temp <= 10:
            return "Kalt"
        elif temp <= 20:
            return "Mild"
        else:
            return "Warm"
    elif season == "Sommer":
        if temp <= 15:
            return "Kalt"
        elif temp <= 25:
            return "Mild"
        else:
            return "Warm"
    elif season == "Herbst":
        if temp <= 10:
            return "Kalt"
        elif temp <= 20:
            return "Mild"
        else:
            return "Warm"

clean_data['Gefühl'] = clean_data.apply(lambda x: temperature_feeling(x['Temperatur'], x['Jahreszeit']), axis=1)

# Dummy-Encoding
gefühl_dummies = pd.get_dummies(clean_data['Gefühl'],prefix='gefühl')
print(len(clean_data))
# Dummy-Variablen anhängen und Originalspalten löschen
clean_data = pd.concat([clean_data,gefühl_dummies], axis=1)
#gefühl_dummies.drop(columns=['Gefühl'], inplace=True)

1493


C:\Users\lorda\AppData\Local\Temp\ipykernel_24476\1434982144.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['Jahreszeit'] = clean_data['Monat'].apply(get_season)
C:\Users\lorda\AppData\Local\Temp\ipykernel_24476\1434982144.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_data['Gefühl'] = clean_data.apply(lambda x: temperature_feeling(x['Temperatur'], x['Jahreszeit']), axis=1)


# Hinzufügen von Feiertagen

In diesem Abschnitt werden Feiertage für jedes Datum im Datensatz hinzugefügt. Dies hilft, Feiertagseffekte zu analysieren und sie als zusätzliche Merkmale in Modellen zu verwenden.

## Schritte der Datenverarbeitung

### 1. **Feiertage abrufen**
Die Funktion `get_feiertage` verwendet die `holidays`-Bibliothek, um alle Feiertage für ein bestimmtes Jahr in Deutschland abzurufen. Das Ergebnis ist eine Liste von Feiertagen.


In [77]:
# **Feiertage hinzufügen**
def get_feiertage(year):
    de_holidays = holidays.Germany(years=year)
    return list(de_holidays.keys())

# Feiertagsinformationen für alle Jahre im Datensatz
feiertage_set = {holiday for year in clean_data['Datum'].dt.year.unique() for holiday in get_feiertage(year)}

# Feiertagsspalte erstellen
clean_data['Feiertag'] = clean_data['Datum'].apply(lambda x: 1 if x.date() in feiertage_set else 0)

# Hinzufügen von besonderen Events (KielLauf, Kieler Triathlon, wichtige Fußballspiele)

In diesem Abschnitt werden besondere Events (wie der KielLauf, der Kieler Triathlon und wichtige Fußballspiele) als zusätzliche Merkmale zum Datensatz hinzugefügt. Diese Events können für die Analyse von Trends oder Anomalien an spezifischen Tagen verwendet werden.

## Schritte der Datenverarbeitung

### 1. **Besondere Ereignisdaten**
Es werden Listen von Datumsangaben für besondere Ereignisse erstellt:

- **Kieler Triathlon:** Ein Ereignis, das jährlich stattfindet (vom Jahr 2013 bis 2019).
- **KielLauf:** Ein weiteres jährliches Ereignis von 2013 bis 2019.
- **Wichtige Fußballspiele:** Ein Satz von bedeutenden Spielen der deutschen Fußball-Nationalmannschaft, darunter Spiele der FIFA-Weltmeisterschaft 2014, der UEFA Euro 2016, des FIFA-Konföderationen-Pokals 2017, der FIFA-Weltmeisterschaft 2018 und der Olympischen Spiele 2016.


In [78]:



# **Besondere Events (KielLauf, Kieler Triathlon) hinzufügen**
kieler_triathlon_daten = pd.to_datetime(['2013-08-04', '2014-08-03', '2015-08-02', '2016-08-07', '2017-08-06', 
                                         '2018-08-05', '2019-08-04'])
kiellauf_daten = pd.to_datetime(["2013-09-08", "2014-09-14", "2015-09-13", "2016-09-11", 
                                 "2017-09-10", "2018-09-09", "2019-09-08"])
important_games_germany = pd.to_datetime([
    # FIFA World Cup 2014
    "2014-07-08",  # Germany 7–1 Brazil (Semifinal)
    "2014-07-13",  # Germany 1–0 Argentina (Final)

    # UEFA Euro 2016
    "2016-07-02",  # Germany 1–1 Italy (6–5 on penalties, Quarterfinal)
    "2016-07-07",  # Germany 0–2 France (Semifinal)

    # FIFA Confederations Cup 2017
    "2017-06-19",  # Germany 3–2 Australia
    "2017-06-22",  # Germany 1–1 Chile
    "2017-06-25",  # Germany 3–1 Cameroon
    "2017-06-29",  # Germany 4–1 Mexico (Semifinal)
    "2017-07-02",  # Germany 1–0 Chile (Final)

    # FIFA World Cup 2018
    "2018-06-17",  # Germany 0–1 Mexico (Group stage)
    "2018-06-27",  # Germany 0–2 South Korea (Group stage, eliminated)

    # Olympic Games 2016 (Men's Football)
    "2016-08-20"   # Germany 1–1 Brazil (4–5 on penalties, Final)
])

clean_data['KielLauf'] = clean_data['Datum'].apply(lambda x: 1 if x.date() in kiellauf_daten.date else 0)
clean_data['Kieler_Triathlon'] = clean_data['Datum'].apply(lambda x: 1 if x.date() in kieler_triathlon_daten.date else 0)
clean_data['Fußball'] = clean_data['Datum'].apply(lambda x: 1 if x.date() in important_games_germany.date else 0)


# PaycheckEffect und Inflationsempfindlichkeit hinzufügen

In diesem Abschnitt wird die **PaycheckEffect**-Spalte erstellt, die angibt, ob ein bestimmter Tag im Zeitraum von Gehaltszahlungen (typischerweise zwischen dem 27. eines Monats und dem 5. des nächsten Monats) liegt. Zusätzlich wird eine **Inflationsempfindlichkeit**-Spalte auf Basis der Warengruppe erstellt, um zu analysieren, wie empfindlich die Produkte gegenüber Inflation sind. 

## Schritte der Datenverarbeitung

### 1. **PaycheckEffect erstellen**
Die PaycheckEffect-Variable wird erstellt, um Tage zu kennzeichnen, an denen Gehaltszahlungen erwartet werden (normalerweise zwischen dem 27. eines Monats und dem 5. des nächsten Monats). Diese Variable hat den Wert `1` an diesen Tagen und `0` an anderen Tagen.


In [79]:

# **PaycheckEffect (Gehaltszahlungen)**
clean_data['PaycheckEffect'] = clean_data['Datum'].apply(
    lambda x: 1 if x.day >= 27 or x.day <= 5 else 0
)

# Inflation sensitivity mapping based on WarenGruppe
sensitivity_mapping = {
    1: "High",   # Brot
    2: "High",   # Brötchen
    3: "Moderate",  # Croissant
    4: "Moderate",  # Konditorei
    5: "Moderate",  # Kuchen
    6: "High"    # Saisonbrot
}

# Map the sensitivity to the WarenGruppe
clean_data["InflationSensitivity"] = clean_data["Warengruppe"].map(sensitivity_mapping)

# Perform dummy encoding for sensitivity levels, excluding 'Low'
dummy_encoded = pd.get_dummies(clean_data["InflationSensitivity"], prefix="Sensitivity").drop(columns=["Sensitivity_Low"], errors="ignore")

# Concatenate the dummy columns with the original DataFrame
clean_data = pd.concat([clean_data, dummy_encoded], axis=1)


# Windkategorie basierend auf Windgeschwindigkeit

In diesem Abschnitt wird eine **Windkategorie** erstellt, die auf der Windgeschwindigkeit basiert. Die Windgeschwindigkeiten werden in drei Kategorien unterteilt:
- **Nicht_windig**: Windgeschwindigkeit < 10 km/h
- **Windig**: Windgeschwindigkeit zwischen 10 und 20 km/h
- **Sehr_windig**: Windgeschwindigkeit > 20 km/h

Nach der Kategorisierung wird die **Dummy-Codierung** angewendet, um die Windkategorien als separate Spalten im Datensatz darzustellen.

## Schritte der Datenverarbeitung

### 1. **Windkategorie bestimmen**
Eine Funktion `wind_kategorie()` wird verwendet, um die Windgeschwindigkeit in die entsprechenden Kategorien zu unterteilen. Diese Funktion wird auf jede Zeile im Datensatz angewendet, um die neue Spalte **Windkategorie** zu erstellen.


In [80]:


# **Windkategorie basierend auf Windgeschwindigkeit**
def wind_kategorie(wind):
    if wind < 10:
        return "Nicht_windig"
    elif wind <= 20:
        return "Windig"
    else:
        return "Sehr_windig" 

clean_data['Windkategorie'] = clean_data['Windgeschwindigkeit'].apply(wind_kategorie)

# Dummy-Variablen für Windkategorien erstellen
wind_dummies = pd.get_dummies(clean_data['Windkategorie'], prefix="Wind")
clean_data = pd.concat([clean_data, wind_dummies], axis=1)




# Dummy-Encoding für Jahreszeit und Wetterkategorie

In diesem Abschnitt wird das **Dummy-Encoding** für die **Jahreszeit** durchgeführt, um die saisonalen Informationen als separate binäre Variablen darzustellen. Jede Jahreszeit (Frühling, Sommer, Herbst, Winter) wird in eine eigene Spalte umgewandelt, die entweder den Wert `1` oder `0` enthält, je nachdem, ob die Zeile zu dieser Jahreszeit gehört.

## Schritte der Datenverarbeitung

### 1. **Dummy-Encoding für Jahreszeit**
Mit der Funktion `pd.get_dummies()` werden für jede mögliche Jahreszeit separate Spalten erstellt. Diese Spalten werden dann zu einem neuen DataFrame hinzugefügt.


In [81]:

# **Dummy-Encoding für Jahreszeit und Wetterkategorie**
jahreszeit_dummies = pd.get_dummies(clean_data['Jahreszeit'], prefix="Jahreszeit")

# Dummy-Variablen anhängen und Originalspalten löschen
clean_data = pd.concat([clean_data, jahreszeit_dummies], axis=1)


# Ferienkennung hinzufügen

In diesem Abschnitt wird eine Spalte hinzugefügt, die angibt, ob ein bestimmtes Datum innerhalb der Schulferien liegt oder nicht. Wenn der Tag innerhalb der Schulferien liegt, wird der Wert `1` zugewiesen, andernfalls `0`. 

## Schritte der Datenverarbeitung

### 1. **Schulferien in Kiel definieren**
Die Schulferien für mehrere Jahre werden als Paare von Start- und Enddaten definiert. Diese Daten umfassen die wichtigsten Ferien (Osterferien, Sommerferien, Herbstferien, Weihnachtsferien).


In [82]:
from datetime import datetime, timedelta
import pandas as pd

ferien_kiel = [
    # Schulferien 2013
    ('2013-01-01', '2013-01-05'),  # Weihnachtsferien
    ('2013-03-25', '2013-04-06'),  # Osterferien
    ('2013-06-24', '2013-08-03'),  # Sommerferien
    ('2013-10-04', '2013-10-19'),  # Herbstferien
    ('2013-12-23', '2014-01-04'),  # Weihnachtsferien

    # Schulferien 2014
    ('2014-04-07', '2014-04-22'),  # Osterferien
    ('2014-07-14', '2014-08-23'),  # Sommerferien
    ('2014-10-20', '2014-11-01'),  # Herbstferien
    ('2014-12-22', '2015-01-06'),  # Weihnachtsferien

    # Schulferien 2015
    ('2015-04-01', '2015-04-18'),  # Osterferien
    ('2015-07-16', '2015-08-29'),  # Sommerferien
    ('2015-10-19', '2015-10-31'),  # Herbstferien
    ('2015-12-23', '2016-01-06'),  # Weihnachtsferien

    # Schulferien 2016
    ('2016-03-24', '2016-04-09'),  # Osterferien
    ('2016-07-25', '2016-09-03'),  # Sommerferien
    ('2016-10-17', '2016-10-29'),  # Herbstferien
    ('2016-12-23', '2017-01-07'),  # Weihnachtsferien

    # Schulferien 2017
    ('2017-04-07', '2017-04-22'),  # Osterferien
    ('2017-07-24', '2017-09-02'),  # Sommerferien
    ('2017-10-16', '2017-10-28'),  # Herbstferien
    ('2017-12-21', '2018-01-06'),  # Weihnachtsferien

    # Schulferien 2018
    ('2018-03-29', '2018-04-13'),  # Osterferien
    ('2018-07-09', '2018-08-18'),  # Sommerferien
    ('2018-10-01', '2018-10-13'),  # Herbstferien
    ('2018-12-21', '2019-01-04'),  # Weihnachtsferien

    # Schulferien 2019
    ('2019-04-04', '2019-04-18'),  # Osterferien
    ('2019-07-01', '2019-08-10'),  # Sommerferien
    ('2019-10-04', '2019-10-19'),  # Herbstferien
    ('2019-12-23', '2020-01-06'),  # Weihnachtsferien

    # Schulferien 2020
    ('2020-03-30', '2020-04-17'),  # Osterferien
    ('2020-06-29', '2020-08-08'),  # Sommerferien
    ('2020-10-05', '2020-10-17'),  # Herbstferien
    ('2020-12-23', '2021-01-06'),  # Weihnachtsferien
]

# Konvertiere Ferien zu einer Liste von Datumsobjekten
ferien_tage = []
for start, end in ferien_kiel:
    start_date = datetime.strptime(start, '%Y-%m-%d')
    end_date = datetime.strptime(end, '%Y-%m-%d')
    while start_date <= end_date:
        ferien_tage.append(start_date)
        start_date += timedelta(days=1)


# Set der Ferientage für schnellen Zugriff
ferien_set = set(ferien_tage)

# Spalte hinzufügen: 1 für Ferien, 0 für keine Ferien
clean_data['Ferien'] = clean_data['Datum'].apply(lambda x: 1 if x in ferien_set else 0)




### Neue Spalte basierend auf Wochentagen erstellen

In diesem Abschnitt werden zwei neue Spalten basierend auf den Wochentagen erstellt:

1. **`Tag_Kategorie`**: Diese Spalte kategorisiert die Tage als "Montag bis Freitag" oder "Wochenende", je nachdem, ob der Wochentag (repräsentiert durch `Wochentag`) zwischen Montag (0) und Freitag (4) liegt oder ob es ein Samstag (5) oder Sonntag (6) ist.
2. **`Wochenende`**: Diese Spalte enthält den Wert `1`, wenn der Wochentag auf ein Wochenende fällt (Samstag oder Sonntag), und `0` für Wochentage (Montag bis Freitag).


In [83]:


# Neue Spalte basierend auf Wochentagen erstellen
clean_data['Tag_Kategorie'] = clean_data['Wochentag'].apply(lambda x: 'Montag_bis_Freitag' if x < 5 else 'Wochenende')
# Neue Spalte für Wochenenden erstellen
clean_data['Wochenende'] = clean_data['Wochentag'].apply(lambda x: 1 if x >= 5 else 0)




## Inflation Daten Subset

In diesem Abschnitt laden wir die Inflationsdaten aus der Datei `inflation.csv` und entfernen bestimmte Spalten, um ein Subset der Daten zu erstellen.

### Einlesen der Daten
Wir lesen die CSV-Datei und geben die ursprünglichen Spalten der Daten aus:


In [84]:
inflation_data= pd.read_csv('DATA/inflation.csv')
print(inflation_data.columns)
subset_inflation = inflation_data.drop(columns=[ 'Verbraucherpreisindex', 'Veränderung_Vorjahresmonat',
       'Veränderung_Vormonat',
       'Inflation_Kategorisierung_vormonat_Neutral',
       'Inflation_Kategorisierung_vormonat_Positiv',
       'Inflation_Kategorisierung_vormonat_Neutral.1',
       'Inflation_Kategorisierung_vormonat_Positiv.1'])
print(subset_inflation.columns)

Index(['Jahr', 'Monat', 'Verbraucherpreisindex', 'Veränderung_Vorjahresmonat',
       'Veränderung_Vormonat', 'Inflation_Kategorisierung',
       'Inflation_Kategorisierung_Neutral',
       'Inflation_Kategorisierung_Positiv',
       'Inflation_Kategorisierung_vormonat_Neutral',
       'Inflation_Kategorisierung_vormonat_Positiv',
       'Inflation_Kategorisierung_vormonat_Neutral.1',
       'Inflation_Kategorisierung_vormonat_Positiv.1'],
      dtype='object')
Index(['Jahr', 'Monat', 'Inflation_Kategorisierung',
       'Inflation_Kategorisierung_Neutral',
       'Inflation_Kategorisierung_Positiv'],
      dtype='object')


## Zusammenführen der Inflationsdaten mit den bereinigten Daten

In diesem Schritt fügen wir die Inflationsdaten, die wir zuvor gefiltert haben, zu den bereinigten Wetterdaten (`clean_data`) hinzu. Wir führen die beiden DataFrames basierend auf den Spalten `Jahr` und `Monat` zusammen.

### Zusammenführen der Daten

Die Zusammenführung erfolgt durch einen linken Join, um alle Zeilen aus `clean_data` zu behalten und die entsprechenden Inflationsdaten hinzuzufügen, wenn verfügbar:


In [85]:


clean_data = pd.merge(clean_data, subset_inflation, on=['Jahr', 'Monat'], how='left')
print(clean_data.columns)


Index(['id', 'Datum', 'Warengruppe', 'Bewoelkung', 'Temperatur',
       'Windgeschwindigkeit', 'Wettercode', 'KielerWoche', 'Wochentag',
       'Monat', 'Jahr', 'Jahreszeit', 'Gefühl', 'gefühl_Kalt', 'gefühl_Mild',
       'gefühl_Warm', 'Feiertag', 'KielLauf', 'Kieler_Triathlon', 'Fußball',
       'PaycheckEffect', 'InflationSensitivity', 'Sensitivity_High',
       'Sensitivity_Moderate', 'Windkategorie', 'Wind_Nicht windig',
       'Wind_Sehr windig', 'Wind_Windig', 'Jahreszeit_Frühling',
       'Jahreszeit_Herbst', 'Jahreszeit_Sommer', 'Jahreszeit_Winter', 'Ferien',
       'Tag_Kategorie', 'Wochenende', 'Inflation_Kategorisierung',
       'Inflation_Kategorisierung_Neutral',
       'Inflation_Kategorisierung_Positiv'],
      dtype='object')


## Dummy-Encoding für Wochentag und Warengruppe

In diesem Schritt erstellen wir Dummy-Variablen für die Spalten `Wochentag` und `Warengruppe`, um diese kategorialen Variablen in numerische Formate zu konvertieren, die für Modelle und Analysen verwendet werden können.

### Dummy-Encoding für den Wochentag

Die `Wochentag`-Spalte wird mithilfe von `pd.get_dummies()` in separate Dummy-Variablen umgewandelt:


In [86]:

# **Dummy-Encoding für Jahreszeit und Wetterkategorie**
wochentag_dummy = pd.get_dummies(clean_data['Wochentag'], prefix="Wochentag")
warrengruppe_dummy = pd.get_dummies(clean_data['Warengruppe'], prefix="Warengruppe")


# Dummy-Variablen anhängen und Originalspalten löschen
clean_data = pd.concat([clean_data, wochentag_dummy,warrengruppe_dummy], axis=1)


## Umbenennen der Spalten für Wochentag und Warengruppe

Nachdem wir die Dummy-Variablen für die `Wochentag`- und `Warengruppe`-Spalten erstellt haben, benennen wir die Spalten um, um eine benutzerfreundlichere Bezeichnung zu erhalten. Dies erleichtert die Interpretation der Daten.

### Umbenennen der Wochentag-Spalten

Die Dummy-Variablen für die Wochentage (von `Wochentag_0` bis `Wochentag_6`) werden entsprechend umbenannt:


In [87]:



# Spalte umbenennen
clean_data.rename(columns={"Wochentag_0": "Montag"}, inplace=True)
clean_data.rename(columns={"Wochentag_1": "Dienstag"}, inplace=True)
clean_data.rename(columns={"Wochentag_2": "Mittwoch"}, inplace=True)
clean_data.rename(columns={"Wochentag_3": "Donnerstag"}, inplace=True)
clean_data.rename(columns={"Wochentag_4": "Freitag"}, inplace=True)
clean_data.rename(columns={"Wochentag_5": "Samstag"}, inplace=True)
clean_data.rename(columns={"Wochentag_6": "Sonntag"}, inplace=True)


# Spalte umbenennen
clean_data.rename(columns={"Warengruppe_1": "Brot"}, inplace=True)
clean_data.rename(columns={"Warengruppe_2": "Brötchen"}, inplace=True)
clean_data.rename(columns={"Warengruppe_3": "Croissant"}, inplace=True)
clean_data.rename(columns={"Warengruppe_4": "Konditorei"}, inplace=True)
clean_data.rename(columns={"Warengruppe_5": "Kuchen"}, inplace=True)
clean_data.rename(columns={"Warengruppe_6": "Saisonbrot"}, inplace=True)


## Umwandlung von booleschen Spalten in numerische Werte und Speichern der bereinigten Daten

Im letzten Schritt des Datenvorbereitungsprozesses identifizieren wir die booleschen Spalten und konvertieren sie in numerische Werte (0 für `False` und 1 für `True`), um sie für die Analyse und Modellierung besser nutzbar zu machen. Anschließend speichern wir die bereinigten Daten in einer CSV-Datei.

### Umwandlung der booleschen Spalten

Zunächst identifizieren wir alle Spalten mit dem Datentyp `bool` und wandeln sie dann in den Datentyp `int` um:


In [88]:
# Identify boolean columns
boolean_columns = clean_data.select_dtypes(include=['bool']).columns

# Convert boolean columns to numeric
clean_data[boolean_columns] = clean_data[boolean_columns].astype(int)


clean_data.to_csv(model_data_pfad, index=False)

## Einlesen der CSV-Datei und Berechnung der NaN-Werte

In diesem Schritt lesen wir eine CSV-Datei ein und berechnen die Gesamtanzahl der `NaN`-Werte im Datensatz. Dies ist wichtig, um sicherzustellen, dass alle fehlenden Werte erkannt und entsprechend behandelt werden können.

### Einlesen der Datei

Zunächst geben wir den Pfad zur Datei an und laden den Inhalt in ein pandas DataFrame:


In [89]:
# Datei einlesen (z. B. eine CSV-Datei)
file_path = 'final_data_test.csv'  # Pfad zur Datei
df = pd.read_csv(file_path)

# Anzahl der NaN-Werte berechnen
anzahl_nan = df.isna().sum().sum()

print(f"Die Datei enthält {anzahl_nan} NaN-Werte.")

Die Datei enthält 0 NaN-Werte.
